In [ ]:
# import library
import numpy as np
import pandas as pd
import itertools
import os
import glob 
# from astropy.stats import sigma_clip

from tqdm import tqdm

from scipy.optimize import minimize
from scipy import optimize

In [ ]:
code_folder = "/kaggle/input/ariel-submission/"
exec(open(code_folder+'feature_engineering.py', 'r').read())
exec(open(code_folder+'preprocess_data.py', 'r').read())
exec(open(code_folder+'combine_data.py', 'r').read())
exec(open(code_folder+'inference.py', 'r').read())
exec(open(code_folder+'postprocessing.py', 'r').read())

In [ ]:
path_folder = "/kaggle/input/ariel-data-challenge-2024/"
CHUNKS_SIZE = 1

In [ ]:
test_adc_info = pd.read_csv(os.path.join(path_folder, 'test_adc_info.csv'))
test_adc_info = test_adc_info.set_index('planet_id')
axis_info = pd.read_parquet(os.path.join(path_folder,'axis_info.parquet'))

index = test_adc_info.index
data_train_AIRS, data_train_FGS = preprocess_data(index, CHUNKS_SIZE, path_folder, test_adc_info, axis_info)

In [ ]:
dataset = combine_data(data_train_AIRS, data_train_FGS)

# inference
all_s = []
for i in tqdm(range(len(dataset))):
    signal = dataset[i,:,1:].mean(axis=1)
    p1, p2 = phase_detector(signal)
    r = minimize(
                objective,
                [0.0001],
                method= 'Nelder-Mead'
                  )
    s = r.x[0]
    all_s.append(s)
    
all_s = np.repeat(np.array(all_s), 283).reshape((len(all_s), 283))        
all_s.shape

In [ ]:
pred = all_s.clip(0)
sigma = np.ones_like(all_s) * 0.000145

sub_df = postprocessing(pred, sigma, index)
sub_df.to_csv('/kaggle/working/submission.csv')